In [1]:
import pandas as pd
from processing import *

from sklearn.metrics.pairwise import cosine_similarity

In [4]:
data=pd.read_csv('/Users/vikiggg/Google/LHL/CarPriceEstimator_Data/vehicles.csv')

In [6]:
cars=fill_nulls(data)

Removed rows lack of information
Removed irreasonable rows which price==0
Filled odometer
Filled default models
revised 10282 car's manufacturers.
droped 8539 cars.
left 289261 cars.
Filled manufacturers
Added a column (manufacturer,model)
Filled partial na by default values of the certain car model
Filled all nulls


In [7]:
cars

,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,price
0,2000.0,GMC,NEW SIERRA 1500,excellent,8 cylinders,gas,167783.00000,clean,automatic,4wd,full-size,truck,red,6995
1,2013.0,HYUNDAI,SONATA,excellent,4 cylinders,gas,90821.00000,clean,automatic,fwd,full-size,sedan,grey,8750
2,2013.0,TOYOTA,PRIUS,good,4 cylinders,hybrid,92800.00000,clean,automatic,fwd,mid-size,hatchback,blue,10900
3,2003.0,MITSUBISHI,LANCER,good,4 cylinders,gas,153672.55064,clean,manual,4wd,mid-size,sedan,grey,12500
4,2007.0,GMC,SIERRA CLASSIC 2500HD,good,8 cylinders,diesel,254217.00000,clean,automatic,4wd,full-size,truck,white,16995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423852,2006.0,HYUNDAI,SONATA,fair,6 cylinders,gas,159980.00000,clean,automatic,fwd,full-size,sedan,blue,1600
423853,2003.0,TOYOTA,SEQUOIA LIMITED,excellent,8 cylinders,gas,160000.00000,clean,automatic,4wd,full-size,SUV,green,9000
423854,1994.0,FORD,F150,fair,6 cylinders,gas,212000.00000,clean,manual,rwd,full-size,truck,green,700
423855,1999.0,LINCOLN,TOWN CAR,excellent,8 cylinders,gas,160000.00000,clean,automatic,rwd,full-size,sedan,white,3800


In [3]:
def fill_nulls(cars):
    cars=cars[~cars['year'].isnull()]
    cars=cars[~((cars['manufacturer'].isnull())&(cars['model'].isnull()))]
    print('Removed rows lack of information')
    cars=cars[~(cars['price']==0)]
    print ('Removed irreasonable rows which price==0')
    cars=Transformer(outlier_age_meters).fit_transform(cars)
    print('Filled odometer')
    cars=fill_default_model(cars)
    print ('Filled default models')
    cars['model']=cars['model'].astype(str).apply(make_lower)
    cars=revise_manu(cars)
    cars['manufacturer']=cars['manufacturer'].apply(rm_punctuation)
    cars['model']=cars['model'].apply(rm_punctuation)
    cars=revise_manufacturers(cars)
    print ('Filled manufacturers')
    cars['car_model'] = list(zip(cars.manufacturer, cars.model))
    print ('Added a column (manufacturer,model)')
    cars=fill_na_by_default(cars)
    print ('Filled partial na by default values of the certain car model')
    cars=fill_freq(cars)
    print ('Filled all nulls')
    return cars

In [14]:
data.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year             95114
manufacturer    110615
model            98473
condition       247138
cylinders       226178
fuel             96643
odometer        153272
title_status     96098
transmission     95792
vin             239437
drive           192738
size            321230
type            182700
paint_color     201654
image_url        94196
description      94203
county          423857
state                0
lat              99453
long             99453
dtype: int64

In [6]:
pickle.dump(data,open(PATH+'processed_data_for_rec.txt','wb'))

In [7]:
for col in cat_features:
    print (data[col].value_counts())

FORD                54126
CHEVROLET           42812
TOYOTA              24002
HONDA               17416
NISSAN              15904
JEEP                13208
GMC                 11804
DODGE               11069
RAM                 10651
HYUNDAI              7886
SUBARU               7643
BMW                  7162
VOLKSWAGEN           6739
MERCEDESBENZ         6477
KIA                  6236
CHRYSLER             5402
BUICK                4402
CADILLAC             4285
MAZDA                3670
LEXUS                3606
AUDI                 3012
ACURA                2595
LINCOLN              2407
PONTIAC              2236
INFINITI             2027
VOLVO                2017
MITSUBISHI           1773
MERCURY              1298
MINI                 1270
ROVER                1101
SATURN               1053
TRUCK                 793
JAGUAR                764
FIAT                  496
ISUZU                 295
TESLA                 284
PORSCHE               202
SCION                 178
SUZUKI      

In [8]:
input_df=pd.DataFrame({'year':2006,
    'manufacturer':'DODGE',
  'model':'POLARA 500',
   'condition':'excellent',
    'cylinders':'8 cylinders',
     'fuel':'gas',
     'odometer': 159980.0,
'title_status':'clean',
 'transmission':'automatic',
  'drive':'rwd',
   'size':'full-size',
    'type':'coupe',
     'paint_color':'white'},index=[0])
input_df.shape

(1, 13)

In [9]:
sample=input_df.copy()
sample=ready_to_predict(sample)

In [10]:
sample

,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
0,90,12,13935,0,6,2,0.31996,0,0,2,1,3,10


In [11]:
price_pred=trained_model.predict(sample)[0]

In [13]:
price_pred

12335.482187361718

In [18]:
rec_features=['year', 'manufacturer',
     'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
     'transmission', 'drive', 'size', 'type', 'paint_color','price']
res_features=['description','price', 'year', 'manufacturer','model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status','transmission','drive', 'size', 'type', 'paint_color','state']

def upper(text):
     return text.upper()

def get_sell_recommendation(input_df,rec_data,pred_price):
     user_input=input_df.copy()
     cars=rec_data[(rec_data['manufacturer']==user_input['manufacturer'][0])&(rec_data['model'].str.contains(user_input['model'][0]))][rec_features].reset_index()

     if cars.shape[0]==0:
          res_df='None'
     else:
          user_input['price']=pred_price
          user_input[label_features]=label.transform(user_input[label_features])
          user_input=user_input[rec_features]
          cars[label_features]=label.transform(cars[label_features])
          cosine_similarities=pd.DataFrame(cosine_similarity(user_input,cars.drop('index',axis=1)))
          top5=cosine_similarities.loc[0].sort_values(ascending=False).head().index.to_list()
          res_df=cars.iloc[top5]
          res_df=res_df.set_index('index')
          lst=res_df.index.to_list()
          res_df=rec_data.loc[lst][res_features]
          try:
               res_df['state']=res_df['state'].apply(upper)
          except:
               res_df['state']='NA'

          res_df.reset_index(drop=True,inplace=True)
     return res_df
          
     

In [21]:
get_sell_recommendation(input_df,data,price_pred)

,description,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,Dodge Polara convertible- hasn’t been run in y...,4900,1968.0,DODGE,POLARA 500,excellent,8 cylinders,gas,68590.546763,missing,automatic,rwd,full-size,coupe,white,NM
1,1964 Dodge Polara 500 2dr hardtop 440 w/727 au...,12500,1964.0,DODGE,POLARA 500,good,8 cylinders,gas,58498.957265,clean,automatic,rwd,full-size,coupe,white,TN


In [24]:
def get_buy_recommendation(user_input,rec_data,pred_price):
    cars=rec_data[rec_features]
    user_input['price']=pred_price
    user_input[label_features]=label.transform(user_input[label_features])
    user_input=user_input[rec_features]
    cars[label_features]=label.transform(cars[label_features])
    cosine_similarities=pd.DataFrame(cosine_similarity(user_input,cars))
    top5=cosine_similarities.loc[0].sort_values(ascending=False).head().index.to_list()
    res_df=rec_data.iloc[top5]
    
    try:
        res_df['state']=res_df['state'].apply(upper)
    except:
        res_df['state']='NA'
   
    res_df.reset_index(drop=True,inplace=True)
    return res_df

In [27]:
get_buy_recommendation(input_df,data,price_pred)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long,age,default_model,car_model,default_drive,default_cylinders,default_fuel,default_transmission,default_size,default_type,default_paint_color,default_title_status
0,7188176986,https://westslope.craigslist.org/ctd/d/grand-j...,western slope,https://westslope.craigslist.org,15494,2004.0,FORD,SUPER DUTY F350 SRW,excellent,8 cylinders,diesel,200512.0,clean,automatic,1FTSW31P24EC99407,4wd,full-size,pickup,white,https://images.craigslist.org/00m0m_dK71QUblfC...,Grand Valley Auto Sales address: 2485 Highw...,NaN,CO,39.0842,-108.5930,16.0,,"(FORD, SUPER DUTY F350 SRW)",4wd,8 cylinders,diesel,automatic,full-size,truck,white,clean
1,7193556133,https://kpr.craigslist.org/ctd/d/kennewick-201...,kennewick-pasco-richland,https://kpr.craigslist.org,8998,2015.0,DODGE,GRAND CARAVAN,excellent,6 cylinders,gas,116617.0,clean,automatic,2C4RDGCG8FR535380,fwd,full-size,mini-van,white,https://images.craigslist.org/00303_5C49sgmonb...,Leskovar Mitsubishi address: 3020 W. Clearw...,NaN,WA,46.2138,-119.1610,5.0,,"(DODGE, GRAND CARAVAN)",fwd,6 cylinders,gas,automatic,full-size,mini-van,white,clean
2,7192910644,https://norfolk.craigslist.org/ctd/d/chesapeak...,norfolk / hampton roads,https://norfolk.craigslist.org,10995,2010.0,ACURA,MDX,excellent,6 cylinders,gas,142805.0,clean,automatic,2HNYD2H20AH527648,4wd,full-size,SUV,black,https://images.craigslist.org/00G0G_UdKsy2Q252...,This Acura MDX can be yours today!Good credit ...,NaN,VA,36.7352,-76.2384,10.0,,"(ACURA, MDX)",4wd,6 cylinders,gas,automatic,full-size,SUV,black,clean
3,7184671926,https://jonesboro.craigslist.org/ctd/d/melbour...,jonesboro,https://jonesboro.craigslist.org,8995,2013.0,DODGE,GRAND CARAVAN,excellent,6 cylinders,gas,116795.0,clean,automatic,2C4RDGCG4DR790833,fwd,full-size,van,grey,https://images.craigslist.org/00n0n_kGnk8F7ZWI...,2013 Dodge Grand Caravan SXT -- Down Payments ...,NaN,AR,36.0584,-91.8792,7.0,,"(DODGE, GRAND CARAVAN)",fwd,6 cylinders,gas,automatic,full-size,mini-van,white,clean
4,7187322791,https://jonesboro.craigslist.org/ctd/d/melbour...,jonesboro,https://jonesboro.craigslist.org,8995,2013.0,DODGE,GRAND CARAVAN,excellent,6 cylinders,gas,116795.0,clean,automatic,2C4RDGCG4DR790833,fwd,full-size,van,grey,https://images.craigslist.org/00n0n_kGnk8F7ZWI...,2013 Dodge Grand Caravan SXT -- Down Payments ...,NaN,AR,36.0584,-91.8792,7.0,,"(DODGE, GRAND CARAVAN)",fwd,6 cylinders,gas,automatic,full-size,mini-van,white,clean
